In [368]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
%matplotlib inline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_validate
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

In [369]:
#откроем данные
data = pd.read_csv('unconv.csv')
data

,Well,Por,Perm,AI,Brittle,TOC,VR,Prod
0,1,12.08,2.92,2.80,81.40,1.16,2.31,4165.196191
1,2,12.38,3.53,3.22,46.17,0.89,1.88,3561.146205
2,3,14.02,2.59,4.01,72.80,0.89,2.72,4284.348574
3,4,17.67,6.75,2.63,39.81,1.08,1.88,5098.680869
4,5,17.52,4.57,3.18,10.94,1.51,1.90,3406.132832
...,...,...,...,...,...,...,...,...
195,196,11.95,3.13,2.97,67.18,0.80,2.06,3847.571003
196,197,17.99,9.87,3.38,44.32,0.98,2.08,5601.227131
197,198,12.12,2.27,3.52,57.07,-0.04,1.73,3409.575363
198,199,15.55,4.48,2.48,58.25,1.89,2.35,5087.592149


## Задание 2.1

In [370]:
#Постройте корреляционную матрицу факторов, включив в неё целевой признак.
corr_matrix = data.corr(method='pearson')
corr_matrix

,Well,Por,Perm,AI,Brittle,TOC,VR,Prod
Well,1.000000,0.068927,0.077928,0.041483,-0.079252,0.022624,-0.007279,0.026817
Por,0.068927,1.000000,0.760546,-0.461549,-0.218570,0.711831,0.111860,0.861910
Perm,0.077928,0.760546,1.000000,-0.239636,-0.124017,0.471746,0.051023,0.727426
AI,0.041483,-0.461549,-0.239636,1.000000,0.127599,-0.531864,0.499143,-0.390835
Brittle,-0.079252,-0.218570,-0.124017,0.127599,1.000000,-0.214282,0.317929,0.237155
TOC,0.022624,0.711831,0.471746,-0.531864,-0.214282,1.000000,0.299483,0.654445
VR,-0.007279,0.111860,0.051023,0.499143,0.317929,0.299483,1.000000,0.323182
Prod,0.026817,0.861910,0.727426,-0.390835,0.237155,0.654445,0.323182,1.000000


In [371]:
#Вычислите ранг полученной матрицы корреляций
np.linalg.matrix_rank(corr_matrix)

8

In [372]:
#Вычислите определитель матрицы корреляций
np.linalg.det(corr_matrix)

0.0007299388072652095

## Задание 2.2

In [373]:
#Создайте матрицу наблюдений. Обозначьте её за X, а вектор правильных ответов — за y .
#Срез факторов без целевого признака
x = data.iloc[:,:-1]            
       
#Добавляем в матрицу стол единиц, для корректной работы формулы
ones = np.ones((data.shape[0],1))     
X = np.hstack((ones, x))

#Вектор целевого
y = data['Prod'].values               

In [374]:
#Постройте модель линейной регрессии по методу наименьших квадратов. 
#Для этого используйте матричную формулу NumPy. 
#В качестве ответа укажите полученные оценки коэффициентов модели
w_hat = np.linalg.inv(X.T@X)@X.T@y   #используя формулу получаем коэффициенты
w = pd.DataFrame(w_hat,index = ['Zeros','Well','Por','Perm','AI','Brittle','TOC','VR'],  columns=['w_hat'])
print('OLS коэффициенты')
display(w)



OLS коэффициенты


,w_hat
Zeros,-1232.308030
Well,0.050700
Por,230.179140
Perm,116.239006
AI,-365.202301
Brittle,24.994370
TOC,-78.400929
VR,785.259815


## Задание 2.3

In [375]:
#Постройте прогноз выработки газа для скважины с параметрами, указанными ниже.
#Чему равна абсолютная ошибка построенного вами прогноза для предложенной скважины ?(в миллионах кубических футов в день).
new_array = np.array([106, 15.32, 3.71, 3.29, 55.99, 1.35, 2.42, 4748.315024])
w_hat_new = np.insert(new_array[:-1], 0, 1)
prediction = w_hat_new@w_hat
print(f'Вектор признаков:{w_hat_new}')
print(f'Прогноз целевого значения по новым данным: {prediction}')
error_new_prod = np.round(np.abs(new_array[-1] - prediction))

print(f'Абсолютная ошибка прогноза для целевого значения новой скважины: {error_new_prod} mcf/day')



Вектор признаков:[  1.   106.    15.32   3.71   3.29  55.99   1.35   2.42]
Прогноз целевого значения по новым данным: 4723.06405370718
Абсолютная ошибка прогноза для целевого значения новой скважины: 25.0 mcf/day


Метрика MAPE:

![](https://miro.medium.com/max/596/1*vmIM1jePXsDxPpgWEQ35Lg.png)

In [376]:
#Постройте прогноз выработки газа для всех скважин из обучающего набора данных. 
#Чему равно значение метрики MAPE вашей модели?
y_predictions = X@w_hat
y_predictions_df = pd.DataFrame(y_predictions,index = range(1,201), columns=['Prod predict'])
MAPE =np.mean((abs(y - y_predictions) / y))*100
MAPE



3.6279468451339723

## Задание 2.5

In [377]:
#Исключите из данных сильно коррелированные между собой факторы.
# Удаляем столбец 'Prod' и считаем корреляционную матрицу
features = data.drop('Prod', axis=1)
features_corr = features.corr()

# Обнуляем диагональные элементы матрицы корреляции
np.fill_diagonal(features_corr.values, 0)

# Вычисляем максимальную абсолютную корреляцию для каждого признака
features_corr_top = features_corr.abs().max().sort_values(ascending=False)

# Выводим результат
print('Топ факторов по коллинеарности:')
display(features_corr_top)

#Список признаков с корреляцией меньше 0.7 между собой
features_corr_below_07 = features_corr_top[features_corr_top < 0.7]

#Топ корреляций к целевому
#Удаляем Prod в строках и смотрим именно столбец Prod, сорируем
high_corr_to_target = corr_matrix.drop('Prod', axis=0)['Prod'].sort_values(ascending=False).abs()

#Выводим резулбтат
print('Топ признаков по корреляции с таргетом:')
display(high_corr_to_target)

#Конечный список признаков при выполнении двух условий: корреляция к таргету > 0.05 и внутренняя корреляция < 0.7
final_features = data.copy().drop(['Perm', 'Well', 'TOC'], axis = 1)
print('Отредактированный набор данных:')
display(final_features)


Топ факторов по коллинеарности:


Por        0.760546
Perm       0.760546
TOC        0.711831
AI         0.531864
VR         0.499143
Brittle    0.317929
Well       0.079252
dtype: float64

Топ признаков по корреляции с таргетом:


Por        0.861910
Perm       0.727426
TOC        0.654445
VR         0.323182
Brittle    0.237155
Well       0.026817
AI         0.390835
Name: Prod, dtype: float64

Отредактированный набор данных:


,Por,AI,Brittle,VR,Prod
0,12.08,2.80,81.40,2.31,4165.196191
1,12.38,3.22,46.17,1.88,3561.146205
2,14.02,4.01,72.80,2.72,4284.348574
3,17.67,2.63,39.81,1.88,5098.680869
4,17.52,3.18,10.94,1.90,3406.132832
...,...,...,...,...,...
195,11.95,2.97,67.18,2.06,3847.571003
196,17.99,3.38,44.32,2.08,5601.227131
197,12.12,3.52,57.07,1.73,3409.575363
198,15.55,2.48,58.25,2.35,5087.592149


In [378]:
final_features.shape

(200, 5)

In [379]:
#Постройте линейную регрессию на обновлённых после удаления факторов данных по методу наименьших квадратов. 
#Срез факторов без целевого признака
x = final_features.iloc[:,:-1].values   
ones = np.ones((final_features.shape[0],1))     
X = np.hstack((ones, x))


#Вектор целевого
y = data['Prod'].values 

#OLS оценка для коэффициентов
w_hat = np.linalg.inv(X.T@X)@X.T@y   #используя формулу получаем коэффициенты
display(w_hat)



array([-1835.44646069,   293.03624565,  -200.03091206,    27.64098209,
         517.40272597])

In [380]:
# Сделайте прогноз для всего обучающего набора данных и рассчитайте метрику MAPE (Mean Absolute Percentage Error).
#Повторно считаем метрику на основе новых данных
y_predictions_new = X@w_hat
y_predictions_new_df = pd.DataFrame(y_predictions_new,index = range(1,201), columns=['Prod predict'])
MAPE = np.mean((abs(y - y_predictions_new) / y))*100
#Выводим результат
display(MAPE)
upd_corr_matrix = final_features.corr()
print('Определитель обновленной матрицы корреляций:')
print(np.round(np.linalg.det(upd_corr_matrix), 4))


4.04413842043617

Определитель обновленной матрицы корреляций:
0.0202


Определитель вырос, а это значит:  
-признаки стали менее коррелированными (снизилась мультиколлинеарность)  
-столбцы стали более линейно независимыми

## Задание 3.1

In [381]:
#Сгенерируйте полиномиальные признаки третьего порядка на факторах, которые вы выбрали для обучения моделей
poly = PolynomialFeatures(degree=3, include_bias=False)
X_poly = poly.fit_transform(x)
#Матрица корреляций
Corr = pd.DataFrame(X_poly).corr()
display(Corr.head())

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,1.000000,-0.461549,-0.218570,0.111860,0.990434,0.538453,0.407119,0.801069,-0.459170,-0.403309,...,-0.451631,-0.451561,-0.320629,-0.345456,-0.274902,-0.119447,-0.226148,-0.166971,-0.050819,0.101129
1,-0.461549,1.000000,0.127599,0.499143,-0.463053,0.476720,-0.162175,-0.053564,0.990349,0.617187,...,0.966394,0.795725,0.928306,0.434131,0.647273,0.765985,0.110745,0.233969,0.390908,0.496688
2,-0.218570,0.127599,1.000000,0.317929,-0.228724,-0.083961,0.784522,0.030386,0.128024,0.843103,...,0.128843,0.643363,0.196447,0.910270,0.745657,0.259970,0.942159,0.940167,0.760297,0.295148
3,0.111860,0.499143,0.317929,1.000000,0.092001,0.574535,0.378253,0.670272,0.494511,0.500412,...,0.485299,0.531966,0.712188,0.417924,0.719467,0.901302,0.261667,0.511891,0.810496,0.972025
4,0.990434,-0.463053,-0.228724,0.092001,1.000000,0.513232,0.381473,0.784334,-0.456294,-0.404766,...,-0.445681,-0.446298,-0.323557,-0.347521,-0.280980,-0.130396,-0.235689,-0.179110,-0.065318,0.086113


In [382]:
#Обучите модель линейной регрессии из библиотеки sklearn (LinearRegression) на полученных полиномиальных факторах.

def polynomial_regression(x, y, k):
    # Создаем полиномиальные признаки заданной степени k
    poly = PolynomialFeatures(degree=k, include_bias=False)
    X_poly = poly.fit_transform(x)

    # Создаем модель линейной регрессии
    lr = LinearRegression()

    # Оцениваем качество модели на кросс-валидации, используя MAPE
    cv_results = cross_validate(lr, X_poly, y, scoring='neg_mean_absolute_percentage_error', cv=5, return_train_score=True)
    
    # Изменяем знак и преобразуем результаты к положительным значениям MAPE
    train_res = -cv_results["train_score"].mean() * 100
    test_res = -cv_results["test_score"].mean() * 100

    return X_poly, train_res, test_res


In [383]:
#Оцените среднее значение метрики MAPE, используя кросс-валидацию на пяти фолдах.
k = 3
X_poly, train_res, test_res = polynomial_regression(x, y, k)
print('--')
print(f'Порядок полинома: {k}')
print(f'MAPE на тренировочных фолдах: {round(train_res, 3)} %')
print(f'MAPE на валидационных фолдах: {round(test_res, 3)} %')

# считаем ранг корреляционной матрицы
print('Ранг корреляционной матрицы:', np.linalg.matrix_rank(Corr))
# считаем количество факторов 
print('Количество факторов:', X_poly.shape[1])

--
Порядок полинома: 3
MAPE на тренировочных фолдах: 1.773 %
MAPE на валидационных фолдах: 2.681 %
Ранг корреляционной матрицы: 34
Количество факторов: 34


## Задание 3.2

In [384]:
# Обучите модель Lasso из библиотеки sklearn на полученных полиномиальных факторах, предварительно стандартизировав факторы с помощью StandardScaler. Коэффициент регуляризации выставите равным 5.

#Оцените среднее значение метрики MAPE, используя кросс-валидацию на пяти фолдах.

def polynomial_standard_regression_l1_sk(x, y, k, alpha):
    # StandardScaler используется для нормализации исходных данных X
    scaler = StandardScaler()
    # Нормализуем данные, вычисляя и применяя среднее и стандартное отклонение
    X_st = scaler.fit_transform(x)

    # Генерируем полиномиальные признаки степени k без добавления свободного члена
    poly = PolynomialFeatures(degree=k, include_bias=False)
    X_poly_st = poly.fit_transform(X_st)

    # Инициализируем модель Lasso для линейной регрессии с L1-регуляризацией
    lasso = Lasso(alpha=alpha)

    # Применяем кросс-валидацию для оценки модели, используя отрицательное значение MAPE
    cv_results = cross_validate(
        lasso, X_poly_st, y, scoring='neg_mean_absolute_percentage_error', cv=5, return_train_score=True)

    # Вычисляем средние значения MAPE на обучающей и тестовой выборках, конвертируя результаты в положительные значения
    train_res = -cv_results["train_score"].mean() * 100
    test_res = -cv_results["test_score"].mean() * 100

    return X_poly_st, train_res, test_res

In [385]:
# Устанавливаем степень полинома
k = 3 
# Устанавливаем коэффициент L1 регуляризации
alpha = 5 
X_poly_st_l1, train_res, test_res = polynomial_standard_regression_l1_sk(x, y, k, alpha)

print('--')
print(f'Порядок полинома: {k}')
print(f'Значение L1 регуляризации: {alpha}')
print(f'MAPE на тренировочных фолдах: {round(train_res, 2)} %')
print(f'MAPE на валидационных фолдах: {round(test_res, 2)} %')

# Вычисление корреляционной матрицы 
D = pd.DataFrame(X_poly_st_l1).corr()
# Определение ранга корреляционной матрицы
print('Ранг корреляционной матрицы:', np.linalg.matrix_rank(D))
# Подсчет количества полиномиальных факторов
print('Количество полиномиальных факторов:', X_poly_st_l1.shape[1])

--
Порядок полинома: 3
Значение L1 регуляризации: 5
MAPE на тренировочных фолдах: 1.83 %
MAPE на валидационных фолдах: 2.28 %
Ранг корреляционной матрицы: 34
Количество полиномиальных факторов: 34


## Задание 3.3

In [386]:
#Обучите модель Ridge из библиотеки sklearn на полученных полиномиальных факторах, предварительно стандартизировав факторы с помощью StandardScaler. Коэффициент регуляризации выставите равным 1.

#Оцените среднее значение метрики MAPE, используя кросс-валидацию на пяти фолдах.
def polynomial_standard_regression_l2(x, y, k, alpha):
    # Стандартизация признаков
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(x)

    # Создание полиномиальных признаков со степенью k
    poly = PolynomialFeatures(degree=k, include_bias=False)
    X_poly = poly.fit_transform(X_scaled)

    # Обучение модели Ridge с заданным коэффициентом регуляризации alpha
    ridge = Ridge(alpha=alpha)

    # Оценка модели с использованием кросс-валидации и MAPE
    cv_results = cross_validate(
        ridge, X_poly, y, scoring='neg_mean_absolute_percentage_error', cv=5, return_train_score=True
    )
    
    # Вычисление среднего значения MAPE
    train_mape = -cv_results['train_score'].mean() * 100
    test_mape = -cv_results['test_score'].mean() * 100

    return train_mape, test_mape


# k = 3
# alpha = 1
# train_mape, test_mape = polynomial_standard_regression_l2(X, y, k, alpha)
# print(f'Train MAPE: {train_mape}%')
# print(f'Test MAPE: {test_mape}%')

In [387]:
k = 3
alpha = 1
train_mape, test_mape = polynomial_standard_regression_l2(x, y, k, alpha)
print(f'MAPE на тренировочных: {round(train_mape,2)}%')
print(f'MAPE на валидационных: {round(test_mape,2)}%')

MAPE на тренировочных: 1.83%
MAPE на валидационных: 2.72%
